In [7]:
# use autograd [https://github.com/HIPS/autograd] to find derivatives
import autograd.numpy as np
from autograd import grad, jacobian
np.set_printoptions(precision=3, suppress=True)

from differentiable_sorting import bitonic_matrices, diff_sort, diff_argsort, softmax, smoothmax
sort_matrices = bitonic_matrices(8)

# test data, length 8
x = [5.0, -1.0, 9.5, 13.2, 16.2, 20.5, 42.0, 18.0]

In [12]:
# uses logsumexp by default
print(diff_sort(sort_matrices, x)) 

[-1.007  4.996  9.439 13.212 15.948 18.21  20.602 42.   ]


In [13]:
# use smoothmax
print(diff_sort(sort_matrices, x, softmax=smoothmax)) 

[-0.955  5.004  9.748 13.009 16.753 17.675 20.167 42.   ]


In [14]:
# we can relax towards averaging, and reverse the ordering by negating alpha 
for alpha in [1.0, 0.75, 0.5, 0.0, -0.5, -1]:
    print(f"Smoothing: {alpha}")    
    print(diff_sort(sort_matrices, x, lambda a,b: smoothmax(a,b, alpha=alpha))) 

Smoothing: 1.0
[-0.955  5.004  9.748 13.009 16.753 17.675 20.167 42.   ]
Smoothing: 0.75
[-0.794  4.947 10.13  12.625 17.126 17.617 19.749 42.   ]
Smoothing: 0.5
[ 0.009  4.576 10.843 11.81  17.578 17.64  18.945 41.999]
Smoothing: 0.0
[15.425 15.425 15.425 15.425 15.425 15.425 15.425 15.425]
Smoothing: -0.5
[41.999 18.945 17.64  17.578 11.81  10.843  4.576  0.009]
Smoothing: -1
[42.    20.167 17.675 16.753 13.009  9.748  5.004 -0.955]


In [19]:
print(np.mean(x))

15.425


In [25]:
###### Ranking
# We can rank as well
x = np.array([1, 2, 3, 4, 8, 7, 6, 4])

print(diff_argsort(sort_matrices, x))


[0. 1. 2. 3. 7. 6. 5. 3.]


In [26]:
# smoothed ranking function
print(diff_argsort(sort_matrices, x, sigma=0.25))

[0.131 1.091 1.999 3.114 6.995 5.997 5.004 3.114]


In [27]:
jac_rank = jacobian(diff_argsort, argnum=1)
print(jac_rank(sort_matrices, np.array(x), 0.25))

[[ 2.162 -1.059 -0.523 -0.287 -0.01  -0.018 -0.056 -0.21 ]
 [-0.066  0.562 -0.186 -0.155 -0.005 -0.011 -0.035 -0.105]
 [-0.012 -0.013  0.041 -0.005 -0.    -0.001 -0.002 -0.008]
 [-0.012 -0.025 -0.108  0.564 -0.05  -0.086 -0.141 -0.14 ]
 [-0.001 -0.001 -0.003 -0.005  0.104 -0.058 -0.028 -0.008]
 [-0.    -0.001 -0.002 -0.004 -0.001  0.028 -0.012 -0.007]
 [-0.    -0.    -0.001 -0.002 -0.016 -0.018  0.038 -0.001]
 [-0.012 -0.025 -0.108 -0.209 -0.05  -0.086 -0.141  0.633]]


In [28]:
# which elements cause the biggest change in ranking if adjusted?
# we can compute this directly:

# 41 and 40 are close to being tied:
x = [41, 2, 30, 40, 50, 60, 70, 190]

# approximate change in rank as first moment of ranks
rank_change = lambda x: np.sum((diff_argsort(sort_matrices, x, sigma=1) * np.arange(1,9)))
grad_rank_change = grad(rank_change)
print("x", np.array(x))
print("d_rank/dx", grad_rank_change(np.array(x)))


x [ 41   2  30  40  50  60  70 190]
d_rank/dx [-0.679 -0.    -0.     0.679 -0.    -0.    -0.     0.   ]
